In [57]:
import numpy as np
import pandas as pd
import argparse
import json

import os
import sys
sys.path.append('./model')

import torch
from torch.utils.data import Dataset, DataLoader
from models.model import MixForecast
from torch.utils.data import ConcatDataset

from tqdm import tqdm
from time import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [65]:
# import torch
# from torch.utils.data import Dataset, ConcatDataset
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# import os

# def standardize_series(series, mean=None, std=None, eps=1e-8):
#     if mean is None or std is None:
#         mean = np.mean(series)
#         std = np.std(series)
#     standardized_series = (series - mean) / (std + eps)
#     return standardized_series, mean, std

# def unscale_predictions(predictions, mean, std, eps=1e-8):
#     return predictions * (std + eps) + mean

# class TimeSeriesDataset(Dataset):
#     def __init__(self, data, backcast_length, forecast_length, stride=1, mean=None, std=None):
#         # Standardize the time series data with provided mean and std
#         self.data, self.mean, self.std = standardize_series(data, mean, std)
#         self.backcast_length = backcast_length
#         self.forecast_length = forecast_length
#         self.stride = stride

#     def __len__(self):
#         return (len(self.data) - self.backcast_length - self.forecast_length) // self.stride + 1

#     def __getitem__(self, index):
#         start_index = index * self.stride
#         x = self.data[start_index : start_index + self.backcast_length]
#         y = self.data[start_index + self.backcast_length : start_index + self.backcast_length + self.forecast_length]
#         return torch.tensor(x, dtype=torch.float32).unsqueeze(0), torch.tensor(y, dtype=torch.float32).unsqueeze(0)


In [66]:
# def load_datasets(folder_path, backcast_length, forecast_length, stride):
#     train_datasets = []
#     val_datasets = []
#     test_datasets = []
    
#     # Initialize mean and std to None, to be computed from train data
    

#     for region in os.listdir(folder_path):
#         region_path = os.path.join(folder_path, region)
#         for building in os.listdir(region_path):

#             mean, std = None, None

#             if building.endswith('.csv'):
#                 file_path = os.path.join(region_path, building)
#                 df = pd.read_csv(file_path)
#                 energy_data = df['energy'].values
                
#                 # Split the energy data into train, val, test with 0.4, 0.1, and 0.5 ratios
#                 train_data, temp_data = train_test_split(energy_data, test_size=0.6, shuffle=False)  # 60% temp_data
#                 val_data, test_data = train_test_split(temp_data, test_size=0.8333, shuffle=False)  # 50% of temp_data for test

#                 # Standardize only the train data and get mean, std
#                 if mean is None or std is None:
#                     train_data, mean, std = standardize_series(train_data)

#                 # Create TimeSeriesDataset for each split using the mean and std from train data
#                 train_dataset = TimeSeriesDataset(train_data, backcast_length, forecast_length, stride, mean, std)
#                 val_dataset = TimeSeriesDataset(val_data, backcast_length, forecast_length, stride, mean, std)
#                 test_dataset = TimeSeriesDataset(test_data, backcast_length, forecast_length, stride, mean, std)

#                 # Append datasets for each split
#                 train_datasets.append(train_dataset)
#                 if not building.startswith('Mathura'):
#                     val_datasets.append(val_dataset)
#                 test_datasets.append(test_dataset)

#     # Combine all datasets for each split
#     combined_train_dataset = ConcatDataset(train_datasets)
#     combined_val_dataset = ConcatDataset(val_datasets)
#     combined_test_dataset = ConcatDataset(test_datasets)

#     return combined_train_dataset, combined_val_dataset, combined_test_dataset

In [67]:



def standardize_series(series, eps=1e-8):
    mean = np.mean(series)
    std = np.std(series)
    standardized_series = (series - mean) / (std+eps)
    return standardized_series, mean, std

def unscale_predictions(predictions, mean, std, eps=1e-8):
    return predictions * (std+eps) + mean


class TimeSeriesDataset(Dataset):
    def __init__(self, data, backcast_length, forecast_length, stride=1):
        # Standardize the time series data
        self.data, self.mean, self.std = standardize_series(data)
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.stride = stride

    def __len__(self):
        return (len(self.data) - self.backcast_length - self.forecast_length) // self.stride + 1

    def __getitem__(self, index):
        start_index = index * self.stride
        x = self.data[start_index : start_index + self.backcast_length]
        y = self.data[start_index + self.backcast_length : start_index + self.backcast_length + self.forecast_length]
        return torch.tensor(x, dtype=torch.float32).unsqueeze(0), torch.tensor(y, dtype=torch.float32).unsqueeze(0)



In [68]:
def load_datasets(folder_path, backcast_length, forecast_length, stride):
    train_datasets = []
    val_datasets = []
    test_datasets = []

    for region in os.listdir(folder_path):
        region_path = os.path.join(folder_path, region)
        for building in os.listdir(region_path):

            if building.endswith('.csv'):
                file_path = os.path.join(region_path, building)
                df = pd.read_csv(file_path)
                energy_data = df['energy'].values
                
                # Split the energy data into train, val, test with 0.4, 0.1, and 0.5 ratios
                train_data, temp_data = train_test_split(energy_data, test_size=0.6, shuffle=False)  # 60% temp_data
                val_data, test_data = train_test_split(temp_data, test_size=0.8333, shuffle=False)  # 50% of temp_data for test

                # Create TimeSeriesDataset for each split
                train_dataset = TimeSeriesDataset(train_data, backcast_length, forecast_length, stride)
                val_dataset = TimeSeriesDataset(val_data, backcast_length, forecast_length, stride)
                test_dataset = TimeSeriesDataset(test_data, backcast_length, forecast_length, stride)

                # Append datasets for each split
                
                train_datasets.append(train_dataset)
                if not building.startswith('Mathura'):
                    val_datasets.append(val_dataset)
                test_datasets.append(test_dataset)

                # train_datasets = [d for d in train_datasets if len(d) > 0]
                # val_datasets = [d for d in val_datasets if len(d) > 0]
                # test_datasets = [d for d in test_datasets if len(d) > 0]

    print(len(train_dataset), len(val_dataset), len(test_dataset))
    # Combine all datasets for each split
    combined_train_dataset = ConcatDataset(train_datasets)
    combined_val_dataset = ConcatDataset(val_datasets)
    combined_test_dataset = ConcatDataset(test_datasets)

    return combined_train_dataset, combined_val_dataset, combined_test_dataset


In [69]:



def train(args, model, criterion, optimizer, device, train_loader, val_loader):

    # Early stopping parameters
    patience = args["patience"]
    best_val_loss = float('inf')
    counter = 0
    early_stop = False

    num_epochs = args["num_epochs"]
    train_start_time = time()  # Start timer

    for epoch in range(num_epochs):

        if early_stop:
            print(f"Early stopping at epoch {epoch + 1}")
            break  

        model.train()
        train_losses = []

        epoch_start_time = time()  # Start epoch timer

        # Progress bar for the training loop
        with tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}', leave=False) as pbar:
            for x_batch, y_batch in pbar:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                backcast, forecast = model(x_batch)
                loss = criterion(forecast, y_batch)
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())

                pbar.set_postfix(loss=loss.item(), elapsed=f"{time() - epoch_start_time:.2f}s")
        
        # Calculate average training loss
        avg_train_loss = np.mean(train_losses)

        # Validation phase
        model.eval()
        val_losses = []
        y_true_val = []
        y_pred_val = []

        # Progress bar for the validation loop
        with tqdm(val_loader, desc=f'Validation Epoch {epoch+1}/{num_epochs}', leave=False) as pbar:
            for x_val, y_val in pbar:
                x_val, y_val = x_val.to(device), y_val.to(device)
                with torch.no_grad():
                    backcast, forecast = model(x_val)
                    loss = criterion(forecast, y_val)
                    val_losses.append(loss.item())
                    
                    # Collect true and predicted values for RMSE calculation
                    y_true_val.extend(y_val.cpu().numpy())
                    y_pred_val.extend(forecast.cpu().numpy())

        # Calculate average validation loss and RMSE
        avg_val_loss = np.mean(val_losses)
        print(len(y_true_val), len(y_pred_val))
        print(y_true_val[0].shape, y_pred_val[0].shape)
        # rmse_val = np.sqrt(mean_squared_error(y_true_val, y_pred_val))

        # Print epoch summary
        print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            counter = 0
            # Save the best model parameters
            os.makedirs(args["finetuned_model_save_path"], exist_ok=True)
            torch.save(model.state_dict(), f'{args["finetuned_model_save_path"]}/best_model.pth')
        else:
            counter += 1
            if counter >= patience:
                early_stop = True


    total_training_time = time() - train_start_time
    print(f'Total Training Time: {total_training_time:.2f}s')



In [ ]:



if __name__ == '__main__':

    # parser = argparse.ArgumentParser(description='Time Series Forecasting')
    # parser.add_argument('--config-file', type=str, default='./configs/model_base.json', help='Input config file path', required=True)
    # file_path_arg = parser.parse_args()
    # config_file = file_path_arg.config_file
    config_file = './configs/model_base_finetune.json'
    with open(config_file, 'r') as f:
        args = json.load(f)


    dataset_path = args["dataset_path"]
 

    # Load datasets
    train_datasets, val_datasets, _ = load_datasets(dataset_path, args["seq_len"], args["pred_len"], args["stride"])

    # Create data loaders
    train_loader = DataLoader(train_datasets, batch_size=args["batch_size"], shuffle=True)
    val_loader = DataLoader(val_datasets, batch_size=args["batch_size"], shuffle=True)

    # check device 
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    num_patches = args["seq_len"] // args["patch_size"]

    # Define MixForecast model
    model = MixForecast(
        device=device,
        forecast_length=args["pred_len"],
        backcast_length=args["seq_len"],
        patch_size = args["patch_size"], 
        num_patches = num_patches,
        num_features = args["num_features"],
        hidden_dim=args["hidden_dim"],
        nb_blocks_per_stack=args["num_blocks_per_stack"],
        stack_layers = args["stack_layers"],
        factor = args["factor"],
    ).to(device)

    model.load_state_dict(torch.load(f'{args["pretrained_model_path"]}/best_model.pth'))
    # model.train()


    # model's parameters
    param = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Model's parameter count is:", param)

    # Define loss and optimizer
    if args["loss"] == "huber":
        criterion = torch.nn.HuberLoss(reduction="mean", delta=1.0)
    else:
        criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())

    # training the model and save best parameters
    train(args=args, model=model, criterion=criterion, optimizer=optimizer, device=device, train_loader=train_loader, val_loader=val_loader)


/tmp/ipykernel_2732082/1066891437.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{args["pretrained_model_path"]}/best_model.pth'))


138 29 174
Model's parameter count is: 116238


174863 174863
(1, 24) (1, 24)
Epoch 1/10, Train Loss: 0.0931, Val Loss: 0.2203


174863 174863
(1, 24) (1, 24)
Epoch 2/10, Train Loss: 0.0925, Val Loss: 0.2231


174863 174863
(1, 24) (1, 24)
Epoch 3/10, Train Loss: 0.0921, Val Loss: 0.2224


174863 174863
(1, 24) (1, 24)
Epoch 4/10, Train Loss: 0.0917, Val Loss: 0.2255
Early stopping at epoch 5
Total Training Time: 126.16s


##### Testing

In [71]:
import numpy as np
import pandas as pd
import json 
from time import time
import argparse

import os
import sys
sys.path.append('./models')

import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from models.model import MixForecast

from tqdm import tqdm
from time import time
from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# metrics used for evaluation
def cal_cvrmse(pred, true, eps=1e-8):
    pred = np.array(pred)
    true = np.array(true)
    return np.power(np.square(pred - true).sum() / pred.shape[0], 0.5) / (true.sum() / pred.shape[0] + eps)

def cal_mae(pred, true):
    pred = np.array(pred)
    true = np.array(true)
    return np.mean(np.abs(pred - true))

def cal_nrmse(pred, true, eps=1e-8):
    true = np.array(true)
    pred = np.array(pred)

    M = len(true) // 24
    y_bar = np.mean(true)
    NRMSE = 100 * (1/ (y_bar+eps)) * np.sqrt((1 / (24 * M)) * np.sum((true - pred) ** 2))
    return NRMSE


def standardize_series(series, eps=1e-8):
    mean = np.mean(series)
    std = np.std(series)
    standardized_series = (series - mean) / (std+eps)
    return standardized_series, mean, std

def unscale_predictions(predictions, mean, std, eps=1e-8):
    return predictions * (std+eps) + mean


class TimeSeriesDataset(Dataset):
    def __init__(self, data, backcast_length, forecast_length, stride=1):
        # Standardize the time series data
        self.data, self.mean, self.std = standardize_series(data)
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.stride = stride

    def __len__(self):
        return (len(self.data) - self.backcast_length - self.forecast_length) // self.stride + 1

    def __getitem__(self, index):
        start_index = index * self.stride
        x = self.data[start_index : start_index + self.backcast_length]
        y = self.data[start_index + self.backcast_length : start_index + self.backcast_length + self.forecast_length]
        return torch.tensor(x, dtype=torch.float32).unsqueeze(0), torch.tensor(y, dtype=torch.float32).unsqueeze(0)


In [72]:


def test(args, model, criterion, device):

    folder_path = args["dataset_path"]
    result_path = args["result_path"]
    
    median_res = []  
    for region in os.listdir(folder_path):

        region_path = os.path.join(folder_path, region)

        results_path = os.path.join(result_path, region)
        os.makedirs(results_path, exist_ok=True)

        res = []

        for building in os.listdir(region_path):

            building_id = building.rsplit(".csv",1)[0]

            if building.endswith('.csv'):
                file_path = os.path.join(region_path, building)
                df = pd.read_csv(file_path)
                energy_data = df['energy'].values

                train_data, temp_data = train_test_split(energy_data, test_size=0.6, shuffle=False)  # 60% temp_data
                val_data, test_data = train_test_split(temp_data, test_size=0.8333, shuffle=False)  # 50% of temp_data for test

                test_dataset = TimeSeriesDataset(test_data, args["seq_len"], args["pred_len"], args["stride"])
                
                # test phase
                model.eval()
                val_losses = []
                y_true_test = []
                y_pred_test = []

                # test loop
                for x_test, y_test in tqdm(DataLoader(test_dataset, batch_size=1), desc=f"Testing {building_id}", leave=False):
                    x_test, y_test = x_test.to(device), y_test.to(device)
                    with torch.no_grad():
                        backcast, forecast = model(x_test)
                        loss = criterion(forecast, y_test)
                        val_losses.append(loss.item())
                        
                        # Collect true and predicted values for RMSE calculation   
                        y_test = y_test.squeeze(1)
                        forecast = forecast.squeeze(1)
                        y_true_test.extend(y_test.cpu().numpy())
                        y_pred_test.extend(forecast.cpu().numpy())
                        
                # Calculate average validation loss and RMSE
                y_true_combine = np.concatenate(y_true_test, axis=0)
                y_pred_combine = np.concatenate(y_pred_test, axis=0)
                avg_test_loss = np.mean(val_losses)
                
                y_pred_combine_unscaled = unscale_predictions(y_pred_combine, test_dataset.mean, test_dataset.std)
                y_true_combine_unscaled = unscale_predictions(y_true_combine, test_dataset.mean, test_dataset.std)
                
                # Calculate CVRMSE, NRMSE, MAE on unscaled data
                cvrmse = cal_cvrmse(y_pred_combine_unscaled, y_true_combine_unscaled)
                nrmse = cal_nrmse(y_pred_combine_unscaled, y_true_combine_unscaled)
                mae = cal_mae(y_pred_combine_unscaled, y_true_combine_unscaled)

                res.append([building_id, cvrmse, nrmse, mae, avg_test_loss])

        columns = ['building_ID', 'CVRMSE', 'NRMSE', 'MAE', 'Avg_Test_Loss']
        df = pd.DataFrame(res, columns=columns)
        df.to_csv("{}/{}.csv".format(results_path, 'result'), index=False)

        med_nrmse = df['NRMSE'].median()
        median_res.append([region, med_nrmse])

    med_columns = ['Dataset','NRMSE']
    median_df = pd.DataFrame(median_res, columns=med_columns)
    median_df.to_csv(f"{result_path}/median_buildings_results.csv", index=False)



In [ ]:



if __name__ == '__main__':

    # parser = argparse.ArgumentParser(description='Time Series Forecasting')
    # parser.add_argument('--config-file', type=str, default='./configs/model_base.json', help='Input config file path', required=True)
    # file_path_arg = parser.parse_args()
    # config_file = file_path_arg.config_file
    config_file = './configs/model_base_finetune.json'
    with open(config_file, 'r') as f:
        args = json.load(f)


    # check device 
    device = args["device"]
    # device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    num_patches = args["seq_len"] // args["patch_size"]
    
    # Define MixForecast model
    model = MixForecast(
        device=device,
        forecast_length=args["pred_len"],
        backcast_length=args["seq_len"],
        patch_size = args["patch_size"], 
        num_patches = num_patches,
        num_features = args["num_features"],
        hidden_dim=args["hidden_dim"],
        nb_blocks_per_stack=args["num_blocks_per_stack"],
        stack_layers = args["stack_layers"],
        factor = args["factor"],
    ).to(device)

    model.load_state_dict(torch.load(f'{args["finetuned_model_save_path"]}/best_model.pth'))

    # model's parameters
    param = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Model's parameter count is:", param)

    # Define loss
    if args["loss"] == "huber":
        criterion = torch.nn.HuberLoss(reduction="mean", delta=1.0)
    else:
        criterion = torch.nn.MSELoss()

    start_time = time()

    # testing the model
    test(args=args, model=model, criterion=criterion, device=device)


    end_time = time() - start_time

    print(f"inference time taken by model is {end_time} sec")




/tmp/ipykernel_2732082/3972630835.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'{args["finetuned_model_save_path"]}/best_model.pth'

Model's parameter count is: 116238


inference time taken by model is 1386.8135697841644 sec
